In [1]:
import requests
import pandas as pd
import time
import random
import concurrent.futures

def fetch_stores(brand_id='sedanos'):
    print("Fetching stores.....")
    headers = {
        'sec-ch-ua-platform': '"Windows"',
        'Referer': 'https://www.sedanos.com/store-locator',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0',
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'sec-ch-ua': '"Microsoft Edge";v="135", "Not-A.Brand";v="8", "Chromium";v="135"',
        'DNT': '1',
        'sec-ch-ua-mobile': '?0',
    }
    
    params = {
        'app_key': 'sedanos',
        'has_address': 'true',
        'limit': '-1',
        'token': 'cb1b0701f3e646bbe5c9e0518addcfc4',
    }
    try:
        response = requests.get('https://api.freshop.com/1/stores', params=params, headers=headers)
        
        stores_json_response = response.json()
        
        stores_dict_list = []
        
        stores_data = stores_json_response['items']
        for store in stores_data:
            if store.get('has_products'):
                store_dict = {
                'brand_id': brand_id,
                'internal_id' : store.get('id', None),
                'address' : store.get('address1', None),
                'city' : store.get('city', None),
                'state': store.get('state', None),
                'zip_code' : store.get('postal_code', None),
                'latitude' : store.get('latitude', None),
                'longitude' : store.get('longitude', None),
                }
                stores_dict_list.append(store_dict) 
        stores_df = pd.DataFrame(stores_dict_list)
        stores_df.drop_duplicates(subset='internal_id', inplace=True)
        print("Finished fetching stores.")
       
        return stores_df
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching stores: {e}")
        return None


In [2]:
# Fetch Categories
def fetch_categories(brand_id=''):
    print("Fetching categories...")
    headers = {
        'accept': 'application/json, text/javascript, */*; q=0.01',
        'accept-language': 'es,es-ES;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
        'dnt': '1',
        'origin': 'https://www.sedanos.com',
        'priority': 'u=1, i',
        'referer': 'https://www.sedanos.com/shop/catering/d/22571166',
        'sec-ch-ua': '"Microsoft Edge";v="135", "Not-A.Brand";v="8", "Chromium";v="135"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'cross-site',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0',
    }
    
    params = {
        'app_key': 'sedanos',
        'custom_filter_id_cascade': 'false',
        'department_id_cascade': 'false',
        'include_custom_filters': 'false',
        'include_departments': 'true',
        'include_offered_together': 'true',
        'include_shelf_tags': 'false',
        'include_tags': 'false',
        'limit': '0',
        'render_id': '1745762834459',
        'token': 'cb1b0701f3e646bbe5c9e0518addcfc4',
    }
    try: 
        response = requests.get('https://api.freshop.com/1/products', params=params, headers=headers)
        
        categories_json_response = response.json()
        
        categories_dict_list = []
        
        categories_data = categories_json_response['departments']
        
        for category in categories_data:
            if (category.get('parent_id', None) == '22394611' and category.get('id') != '22394619'):
                category_dict = {
                'category_id': category['id'],
                'category_name': category['name'],
                'total_products': int(category['count']),
                'brand_id': brand_id
                }
                categories_dict_list.append(category_dict)
            
        categories_df = pd.DataFrame(categories_dict_list)
        categories_df.drop_duplicates(subset='category_id', inplace=True)
        print("Finished fetching categories.")
        return categories_df
    except requests.exceptions.RequestException as e:
        print(f"Error fetching categories: {e}")
        return None

In [3]:
# Fetch Products

def fetch_products_in_category(row, store_id, token='cb1b0701f3e646bbe5c9e0518addcfc4'):
    cat_id = row['category_id']
    
    cat_name = row['category_name']
    LIMIT = 100  
    headers = {
        'accept': 'application/json, text/javascript, */*; q=0.01',
        'accept-language': 'es,es-ES;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
        'dnt': '1',
        'origin': 'https://www.sedanos.com',
        'priority': 'u=1, i',
        'referer': f'https://www.sedanos.com/shop/{cat_name}/d/{cat_id}',
        'sec-ch-ua': '"Microsoft Edge";v="135", "Not-A.Brand";v="8", "Chromium";v="135"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'cross-site',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0',
    }

    all_products = []
    skip = 0

    while True:
        params = {
            'app_key': 'sedanos',
            'department_id': cat_id,
            'department_id_cascade': 'true',
            'fields': 'id, identifier, upc, name, store_id, department_id, size,popularity, canonical_url, varieties,quantity_size_ratio_description,status,status_id,other_attributes,average_rating,review_count,unit_price,offer_sale_price',  
            'include_offered_together': 'true',
            'limit': str(LIMIT),
            'popularity_sort': 'asc',
            'render_id': '1745763999375',
            'sort': 'popularity',
            'store_id': store_id,
            'token': token,
            'skip': str(skip),
        }

        try:
            response = requests.get('https://api.freshop.com/1/products', params=params, headers=headers)
            response.raise_for_status()
            data = response.json()
            products = data.get('items', [])
            all_products.extend(products)
            print(f'✅ Obtenidos {len(products)} productos de la categoría {cat_name} con skip {skip} en {store_id}')
        except Exception as e:
            print(f'❌ Error al obtener productos de la categoría {cat_name} con skip {skip} en {store_id}: {e}')
            break

        
        if len(products) < LIMIT:
            print(f'🛑 Ya no hay más productos para la categoría {cat_name} en {store_id}')
            break

        skip += LIMIT

        sleep_time = random.randint(15, 35)
        print(f'⏳ Esperando {sleep_time}s antes del próximo request...')
        time.sleep(sleep_time)

    return all_products



def fetch_all_item_info(df_categories, max_workers=5):
    print('Fetching items info...')
    item_info_list = []
    # Crea el grupo de hilos
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        #Crea las tareas par cada categoria
        futures = [executor.submit(fetch_products_in_category, row, '5761') for _, row in df_categories.iterrows()]
        
        for i, future in enumerate(concurrent.futures.as_completed(futures)):
            try:
                result = future.result()
                item_info_list.extend(result)
                print(f'Completed {i+1}/{len(futures)}')
            except Exception as e:
                print(f"Error in thread: {e}")

    print('Finished fetching item info.')
    return item_info_list



def parse_products_data(responses_list):
    print('Starting products-parsing...')
    item_dict_list = []
    for product in responses_list:
        name = product.get('name', None)
        id = product.get('id', None)
        upc = product.get('upc', None)
        category_id = product.get('department_id', None)
        size = product.get('size', None)
        
        

        item_dict = {
            'internal_id': id,
            'name': name,
            'upc': upc,
            'category_id': category_id,
            'size': size,
            
            
        }
        
        item_dict_list.append(item_dict)
        

    print(f'Parsed {len(item_dict_list)} items.')
    return item_dict_list

In [4]:
def fetch_prices_for_stores(df_categories, store_ids, max_workers=5):
    prices_list = []
    tasks = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        for store_id in store_ids:
            print(f'Fetching prices for store {store_id}...')
            for _, row in df_categories.iterrows():
                tasks.append(executor.submit(fetch_products_in_category, row, store_id))

        for i, future in enumerate(concurrent.futures.as_completed(tasks)):
            try:
                result = future.result()
                prices_list.extend(result)
                print(f'Completed {i+1}/{len(tasks)}')
            except Exception as e:
                print(f"Error in thread: {e}")

    print('Finished fetching prices for all stores.')
    return prices_list



def parse_prices_data(responses_list):
    print('Starting prices-parsing...')
    prices_dict_list = []
    for product in responses_list:
        product_id = product.get('id', None)
        unit_price = product.get('unit_price', None)
        sale_price = product.get('offer_sale_price', None)
        status = product.get('status', None)
        store_id = product.get('store_id', None)
        item_dict = {
            'product_id': product_id,
            'status': status,
            'unit_price': unit_price,
            'sale_price': sale_price,
            'store_id': store_id,
            'date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
        }
        
        prices_dict_list.append(item_dict)
        

    print(f'Parsed {len(prices_dict_list)} items.')
    return prices_dict_list


In [5]:
def main():
    # Fetch stores
    df_stores = fetch_stores()
    if df_stores is not None:
        df_stores.to_csv('sedanos_stores.csv', index=False)
        print("Stores data saved to stores.csv")

    # Fetch categories
    df_categories = fetch_categories()
    if df_categories is not None:
        df_categories.to_csv('sedanos_categories.csv', index=False)
        print("Categories data saved to categories.csv")

    # Fetch products
    item_info_list = fetch_all_item_info(df_categories, 5)
    item_dict_list = parse_products_data(item_info_list)
    df_items = pd.DataFrame(item_dict_list)
    df_items.to_csv('sedanos_products.csv', index=False)
    
main()

Fetching stores.....
Finished fetching stores.
Stores data saved to stores.csv
Fetching categories...
Finished fetching categories.
Categories data saved to categories.csv
Fetching items info...
✅ Obtenidos 100 productos de la categoría Dairy con skip 0 en 5761
⏳ Esperando 35s antes del próximo request...
✅ Obtenidos 100 productos de la categoría Bakery con skip 0 en 5761
⏳ Esperando 22s antes del próximo request...
✅ Obtenidos 100 productos de la categoría Deli con skip 0 en 5761
⏳ Esperando 31s antes del próximo request...
✅ Obtenidos 100 productos de la categoría Frozen Foods con skip 0 en 5761
⏳ Esperando 16s antes del próximo request...
✅ Obtenidos 100 productos de la categoría Beer, Wine & Spirits con skip 0 en 5761
⏳ Esperando 17s antes del próximo request...
✅ Obtenidos 100 productos de la categoría Frozen Foods con skip 100 en 5761
⏳ Esperando 30s antes del próximo request...
✅ Obtenidos 100 productos de la categoría Beer, Wine & Spirits con skip 100 en 5761
⏳ Esperando 21s an

In [ ]:
def prices():
    # Fetch prices
    stores_ids = ['5761', '5760', '5759']
    df_categories = fetch_categories()
    prices_info_list = fetch_prices_for_stores(df_categories, stores_ids, 5)
    prices_dict_list = parse_prices_data(prices_info_list)
    df_prices = pd.DataFrame(prices_dict_list)
    df_prices.to_csv('sedanos_precios.csv', index=False)
    
prices()
    

Fetching categories...
Finished fetching categories.
Fetching prices for store 5761...
Fetching prices for store 5760...
Fetching prices for store 5759...
✅ Obtenidos 100 productos de la categoría Bakery con skip 0 en 5761
⏳ Esperando 22s antes del próximo request...
✅ Obtenidos 100 productos de la categoría Dairy con skip 0 en 5761
⏳ Esperando 33s antes del próximo request...
✅ Obtenidos 100 productos de la categoría Deli con skip 0 en 5761
⏳ Esperando 32s antes del próximo request...
✅ Obtenidos 100 productos de la categoría Frozen Foods con skip 0 en 5761
⏳ Esperando 16s antes del próximo request...
✅ Obtenidos 100 productos de la categoría Beer, Wine & Spirits con skip 0 en 5761
⏳ Esperando 27s antes del próximo request...
